#### New to Plotly?
Plotly's Python library is free and open source! [Get started](https://plot.ly/python/getting-started/) by downloading the client and [reading the primer](https://plot.ly/python/getting-started/).
<br>You can set up Plotly to work in [online](https://plot.ly/python/getting-started/#initialization-for-online-plotting) or [offline](https://plot.ly/python/getting-started/#initialization-for-offline-plotting) mode, or in [jupyter notebooks](https://plot.ly/python/getting-started/#start-plotting-online).
<br>We also have a quick-reference [cheatsheet](https://images.plot.ly/plotly-documentation/images/python_cheat_sheet.pdf) (new!) to help you get started!

#### Version Check
Plotly's python package is updated frequently. Run `pip install plotly --upgrade` to use the latest version.

In [1]:
import plotly
plotly.__version__

'3.6.0'

#### NYC Flights Database

In [2]:
import datetime
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.plotly as py

from ipywidgets import widgets

We'll be making an application to take a look at delays from all flights out of NYC in the year 2013.

In [3]:
df = pd.read_csv(
    'https://raw.githubusercontent.com/yankev/testing/master/datasets/nycflights.csv')
df = df.drop(df.columns[[0]], axis=1)

In [4]:
df.sample(3)

,year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute
294791,2013,8,16,1306.0,-4.0,1555.0,19.0,FL,N607AT,348,LGA,ATL,124.0,762,13.0,6.0
314484,2013,9,6,1801.0,12.0,2024.0,-28.0,UA,N555UA,731,JFK,LAX,299.0,2475,18.0,1.0
250440,2013,6,30,NaN,NaN,NaN,NaN,EV,N716EV,5599,LGA,CHS,NaN,641,NaN,NaN


Let's get the set of all the `airlines`, so that we can type the right things into the search box later. 

In [5]:
df['carrier'].unique()

array(['UA', 'AA', 'B6', 'DL', 'EV', 'MQ', 'US', 'WN', 'VX', 'FL', 'AS',
       '9E', 'F9', 'HA', 'YV', 'OO'], dtype=object)

Let's assign the widgets that we're going to be using in our app. In general all these widgets will be used to filter the data set, and thus what we visualize.

In [6]:
month = widgets.IntSlider(
    value=1.0,
    min=1.0,
    max=12.0,
    step=1.0,
    description='Month:',
    continuous_update=False
)

use_date = widgets.Checkbox(
    description='Date: ',
    value=True,
)

container = widgets.HBox(children=[use_date, month])

textbox = widgets.Dropdown(
    description='Airline:   ',
    value='DL',
    options=df['carrier'].unique().tolist()
)

origin = widgets.Dropdown(
    options=list(df['origin'].unique()),
    value='LGA',
    description='Origin Airport:',
)


# Assign an emptry figure widget with two traces
trace1 = go.Histogram(x=df['arr_delay'], opacity=0.75, name='Arrival Delays')
trace2 = go.Histogram(x=df['dep_delay'], opacity=0.75, name='Departure Delays')
g = go.FigureWidget(data=[trace1, trace2],
                    layout=go.Layout(
                        title=dict(
                            text='NYC FlightDatabase'
                        ),
                        barmode='overlay'
                    ))

Let now write a function that will handle the input from the widgets, and alter the state of the graph.

In [7]:
def validate():
    if origin.value in df['origin'].unique() and textbox.value in df['carrier'].unique():
        return True
    else:
        return False


def response(change):
    if validate():
        if use_date.value:
            filter_list = [i and j and k for i, j, k in
                           zip(df['month'] == month.value, df['carrier'] == textbox.value,
                               df['origin'] == origin.value)]
            temp_df = df[filter_list]

        else:
            filter_list = [i and j for i, j in
                           zip(df['carrier'] == 'DL', df['origin'] == origin.value)]
            temp_df = df[filter_list]
        x1 = temp_df['arr_delay']
        x2 = temp_df['dep_delay']
        with g.batch_update():
            g.data[0].x = x1
            g.data[1].x = x2
            g.layout.barmode = 'overlay'
            g.layout.xaxis.title = 'Delay in Minutes'
            g.layout.yaxis.title = 'Number of Delays'


origin.observe(response, names="value")
textbox.observe(response, names="value")
month.observe(response, names="value")
use_date.observe(response, names="value")

Time to try the app out!!

In [8]:
container2 = widgets.HBox([origin, textbox])
widgets.VBox([container,
              container2,
              g])

In [9]:
%%html
<img src = 'https://cloud.githubusercontent.com/assets/12302455/16637308/4e476280-43ac-11e6-9fd3-ada2c9506ee1.gif' >

#### Reference

In [10]:
help(go.FigureWidget)

Help on class FigureWidget in module plotly.graph_objs._figurewidget:

class FigureWidget(plotly.basewidget.BaseFigureWidget)
 |  FigureWidget(*args, **kwargs)
 |  
 |  Base class for FigureWidget. The FigureWidget class is code-generated as a
 |  subclass
 |  
 |  Method resolution order:
 |      FigureWidget
 |      plotly.basewidget.BaseFigureWidget
 |      plotly.basedatatypes.BaseFigure
 |      ipywidgets.widgets.domwidget.DOMWidget
 |      ipywidgets.widgets.widget.Widget
 |      ipywidgets.widgets.widget.LoggingHasTraits
 |      traitlets.traitlets.HasTraits
 |      traitlets.traitlets.HasDescriptors
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, data=None, layout=None, frames=None, skip_invalid=False)
 |      Create a new FigureWidget instance
 |      
 |      Parameters
 |      ----------
 |      data
 |          The 'data' property is a tuple of trace instances
 |          that may be specified as:
 |            - A list or tuple of trace insta

In [13]:
from IPython.display import display, HTML

display(HTML('<link href="//fonts.googleapis.com/css?family=Open+Sans:600,400,300,200|Inconsolata|Ubuntu+Mono:400,700" rel="stylesheet" type="text/css" />'))
display(HTML('<link rel="stylesheet" type="text/css" href="http://help.plot.ly/documentation/all_static/css/ipython-notebook-custom.css">'))

!pip install git+https://github.com/plotly/publisher.git --upgrade

import publisher
publisher.publish(
    'figurewidget_app.ipynb', 'python/figurewidget-app/', 'IPython FigureWidget',
    'Interactive Data Analysis with Plotly',
    title='Interactive Data Analysis with FigureWidget ipywidgets',
    name='Interactive Data Analysis with FigureWidget ipywidgets',
    has_thumbnail='true', thumbnail='thumbnail/multi-widget.jpg',
    language='python', page_type='example_index',
    display_as='chart_events', order=23,
    ipynb='~notebook_demo/231')

  Cloning https://github.com/plotly/publisher.git to c:\users\priyat~1\appdata\local\temp\pip-req-build-x22u5hdq
  Running setup.py bdist_wheel for publisher: started
  Running setup.py bdist_wheel for publisher: finished with status 'done'
  Stored in directory: C:\Users\PRIYAT~1\AppData\Local\Temp\pip-ephem-wheel-cache-_fe6341n\wheels\99\3e\a0\fbd22ba24cca72bdbaba53dbc23c1768755fb17b3af0f33966
Successfully built publisher
  Found existing installation: publisher 0.13
    Uninstalling publisher-0.13:
      Successfully uninstalled publisher-0.13
